## Jetson Rover

El proyecto de investigaciÓn MODELADO Y CONTROL DE SISTEMAS NO HOLÓNOMOS CON APLICACIONES A VEHICULOS AUTÓNOMOS, 20230170, se realizó mediante el uso e implementacion de una placa de desarrollo Jetson Nano de 4GB de Nvidia.

![Jetson Nano](../img/Jetson.jpg)

## Configuracion de entorno

Una vez instalammos el sistema operativo proporcionado por Nvidia en su pagina oficial https://developer.nvidia.com/embedded/learn/getting-started-jetson, comenzaremos con la configuracion del entorno de desarrollo para hacer uso de la placa.

Ejecutaremos las siguientes lineas en la terminal de linux (Sistema operativo de la placa):

```
    sudo apt-get update & upgrade
    sudo apt-get install python3.8 
    sudo apt-get install python3-pip python3.8-pip
```


Una vez tengamos listo el lenguaje de programacion haremos la creacion de el entorno virtual que nos permitira trabajar de una manera mas eficiente dentro de python, ademas de que dentro del entorno virtual instalaremos los paquetes y librerias necesarios.

Comenzamos creando el directorio de nuestro proyecyo en el Desktop de la Jetson

```
    sudo apt-get install python3-venv python3.8-venv
    cd Desktop/
    mkdir Rover_Project
    cd Rover_Project/
```

Ahora crearemos el entorno virtual mediante el siguiente comando:

```
    python3.8 -m venv Jetson_Venv
```

Una vez creado el entorno virtual procederemos a activarlo e instalar las dependencias necesarias 

```
    source Jetson_Venv/bin/activate
    pip install --upgrade pip
    pip install numpy 
    pip install matplotlib
    pip install opencv-python
    pip install Jetson.GPIO 
    pip install pyrealsense2
```

## Codigo del rover

Con el entorno completamente configurado procederemos a programar tanto la camara Real sense de intel como la Jetson Nano.

Comenzaremos importanto las librerias

In [2]:
import numpy as np
import cv2 
import pyrealsense2 as rs 
import matplotlib.pyplot as plt

In [ ]:
import Jetson.GPIO as GPIO

Una vez tengamos todas las librerias instaladas crearemos una clase para hacer un uso eficiente de la camara realsense

In [3]:
class RoverCamera:
    def __init__(self) -> None:
        #Config depth and color streams 
        self.pipeline = rs.pipeline()
        config = rs.config()
        
        #Get device and product line settings for better support
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
        
        # Start streaming
        self.pipeline.start(config)
        
    def get_frames(self):
        frames = self.pipeline.wait_for_frames()

        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_time = depth_frame.get_timestamp()

        # Create colormap to show the depth of the Objects
        colorizer = rs.colorizer()
        depth_colormap = np.asanyarray(colorizer.colorize(depth_frame).get_data())


        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_frame, depth_image, color_image, depth_colormap

    def release(self):
        self.pipeline.stop()

Ahora configuraremos la Jetson nano mediante otra clase, que realizara el pwm y el contro proporcional de ambos motores DC

In [ ]:
class Jetson_Rover:
    camera_Rover = RoverCamera()
    distances = []
    point = (320,240)
    velocidad = 0
    ENA = 33
    motorA_1 = 35
    motorA_2 = 37

    ENB = 32
    motorB_1 = 38
    motorB_2 = 36
    
    def __init__(self):
        pass
    def Voltage(self, error):
        V = 100*error
        if abs(V)>=100:
            return 100
        elif abs(V)<100:
            return V

    def Direction(self, error):
        if error>0:
            return (GPIO.HIGH, GPIO.LOW)
        elif error<0:
            return (GPIO.LOW, GPIO.HIGH)

    def draw_Point(self, bgr_frame, depth_frame, distance):
        cv2.circle(bgr_frame,(self.point[0], self.point[1]),2,(0,255,0),2)
        cv2.putText(bgr_frame,"{:.3f}m".format(distance), (self.point[0], self.point[1]-10),1,2,(0,255,0),2)
        
        cv2.circle(depth_frame,(self.point[0], self.point[1]),2,(0,255,0),2)
        cv2.putText(depth_frame,"{:.3f}m".format(distance), (self.point[0], self.point[1]-10),1,2,(0,255,0),2)
        
    def init_Rover(self):
        print("Iniciando Rover...")
        
        pwm_ENA = GPIO.PWM(self.ENA, 500)
        pwm_ENB = GPIO.PWM(self.ENB, 500)
	
        pwm_ENA.start(0)
        pwm_ENB.start(0)
        
        while(True):
            state, depth_frame, depth_image, color_image, depth_colorMap = self.camera_Rover.get_frames()
            
            if not state:
                print("An error have ocurred, verify the camera conection")
                break

            distance = depth_frame.get_distance(self.point[0], self.point[1])
            self.distances.append(distance)
            E = distance-0.2
            direction = self.Direction(E)
            V = self.Voltage(E)
            
            GPIO.output(self.motorA_1, direction[0])
            GPIO.output(self.motorA_2, direction[1])
            GPIO.output(self.motorB_1, direction[0])
            GPIO.output(self.motorB_2, direction[1])
            
            pwm_ENA.ChangeDutyCycle(V)
            pwm_ENB.ChangeDutyCycle(V)

            self.draw_Point(color_image, depth_colorMap, distance)
            
            cv2.imshow("Color frame", color_image)
            cv2.imshow("ColorMap", depth_colorMap)
            
            key = cv2.waitKey(1)
            
            if key == 27:
                pwm_ENA.stop()
                pwm_ENB.stop()
                GPIO.cleanup()
                cv2.destroyAllWindows()
                break


rover = Jetson_Rover()
rover.init_Rover()